In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Just a little fun and an excuse to use constraint programming to find out how many samples there might be in the public LB.

In [ ]:
!wget https://www.kaggle.com/c/33102/publicleaderboarddata.zip

In [ ]:
lb_df = pd.read_csv('./publicleaderboarddata.zip')
scores = np.round((lb_df['Score']*1e5)).astype(np.int64).unique()
scores

In [ ]:
len(scores)

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
# # assume scores were rounded to 5 decimal places
# model = cp_model.CpModel()
# N=32
# x = [model.NewIntVar(0,20000, F'x_{i}') for i in range(len(scores[:N]))]
# n = model.NewIntVar(15000,20000,'n')
# for i in range(len(scores[:N])):
#     model.Add(200000*x[i] >= (2*scores[i]-1)*n)
#     model.Add(200000*x[i] <= (2*scores[i]+1)*n)
    

In [ ]:
# assume scores were truncated to 5 decimal places
model = cp_model.CpModel()
x = [model.NewIntVar(0,20000, F'x_{i}') for i in range(len(scores))]
n = model.NewIntVar(15000,20000,'n')
for i in range(len(scores)):
    model.Add(100000*x[i] >= scores[i]*n)
    model.Add(100000*x[i] < (scores[i]+1)*n)

In [ ]:
solver = cp_model.CpSolver()
solver.SearchForAllSolutions(model,cp_model.VarArraySolutionPrinter([n]))
solver.StatusName()